# Version 2: Notes
* This is a first copy, this version only includes Models 1-8, and is missing Ash's implementation
* Testing = Augmented (34000)
* Training = Original (smaller set 6400)

* All images resized to 128x128
* Pixels were not normalized 
* Had high accuracies, but very low f1 scores

* Issue might be because training set is so much smaller?



In [14]:
! pip install tensorflow
! pip install kagglehub
! pip install psutil
! pip install matplotlib
%pip install scikit-learn
%pip install imbalanced-learn
! pip install pandas



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# Import Libraries and Dataset

In [15]:
# Importing all necessary libraries
import os
import kagglehub

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


# Importing the data to be used in this CNN

In [16]:
path = kagglehub.dataset_download("uraninjo/augmented-alzheimer-mri-dataset")
print("Path to dataset files:", path)

Path to dataset files: /home/gitpod/.cache/kagglehub/datasets/uraninjo/augmented-alzheimer-mri-dataset/versions/1


# Splitting Data to Training and Testing set

In [17]:
# Set the base path
train_dir = os.path.join(path, 'OriginalDataset')
test_dir = os.path.join(path, 'AugmentedAlzheimerDataset')

print("Train Directory:", train_dir)
print("Test Directory:", test_dir)

Train Directory: /home/gitpod/.cache/kagglehub/datasets/uraninjo/augmented-alzheimer-mri-dataset/versions/1/OriginalDataset
Test Directory: /home/gitpod/.cache/kagglehub/datasets/uraninjo/augmented-alzheimer-mri-dataset/versions/1/AugmentedAlzheimerDataset


# Pre-Processing the data (distribution of the data)

In [18]:
# count the files inside each class directory
train_files = sum([len(files) for _, _, files in os.walk(train_dir)])
test_files = sum([len(files) for _, _, files in os.walk(test_dir)])

print(f"Train files count: {train_files}")
print(f"Test files count: {test_files}")

Train files count: 6400
Test files count: 33984


# Splitting the data into Training and Testing dataset

In [19]:
# Load datasets using categorical labels
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(128, 128),          # resize all images to 128x128 pixels
    batch_size=32,                  # load images in batches of 32 for efficient processing
    label_mode='categorical',       # assign categorical labels to each image (one-hot encoded)
    shuffle=True,                   # shuffle the dataset to promote model generalization
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(128, 128),
    batch_size=32,
    label_mode='categorical',
    shuffle=False,                  # do not shuffle to maintain consistent evaluation order
)

Found 6400 files belonging to 4 classes.


Found 33984 files belonging to 4 classes.


# Model 1: Basic CNN

In [21]:
# Model 1: Basic CNN
model_1 = Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(128, 128, 3)),  
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(4, activation='softmax')
])
model_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_1 = model_1.fit(train_dataset, validation_data=val_dataset, epochs=10)


Epoch 1/10


200/200 ━━━━━━━━━━━━━━━━━━━━ 40s 188ms/step - accuracy: 0.4493 - loss: 5.5668 - val_accuracy: 0.6156 - val_loss: 0.8510
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 28s 138ms/step - accuracy: 0.5795 - loss: 0.8811 - val_accuracy: 0.5875 - val_loss: 0.8996
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 28s 138ms/step - accuracy: 0.6523 - loss: 0.7786 - val_accuracy: 0.7563 - val_loss: 0.5977
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 27s 136ms/step - accuracy: 0.7301 - loss: 0.6306 - val_accuracy: 0.8359 - val_loss: 0.4357
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 28s 140ms/step - accuracy: 0.7925 - loss: 0.4823 - val_accuracy: 0.9203 - val_loss: 0.2635
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 35s 173ms/step - accuracy: 0.8584 - loss: 0.3608 - val_accuracy: 0.9062 - val_loss: 0.2105
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 27s 134ms/step - accuracy: 0.8859 - loss: 0.2848 - val_accuracy: 0.9250 - val_loss: 0.2017
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 26s 130ms/step - accuracy: 0.9261 - loss: 0.1962 - val

# Model 2: CNN + L2 Regularization

In [22]:
# Model 2: CNN + L2 Regularization
model_2 = Sequential()
model_2.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(128, 128, 3)))
model_2.add(layers.BatchNormalization())
model_2.add(layers.MaxPooling2D((2, 2)))

model_2.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model_2.add(layers.BatchNormalization())
model_2.add(layers.MaxPooling2D((2, 2)))

model_2.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model_2.add(layers.BatchNormalization())
model_2.add(layers.MaxPooling2D((2, 2)))

model_2.add(layers.Flatten())
model_2.add(layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model_2.add(layers.BatchNormalization())
model_2.add(layers.Dropout(0.5))

model_2.add(layers.Dense(4, activation='softmax'))

model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_2 = model_2.fit(train_dataset, validation_data=val_dataset, epochs=10)


Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 124s 604ms/step - accuracy: 0.5104 - loss: 1.7737 - val_accuracy: 0.5953 - val_loss: 1.4019
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 153s 657ms/step - accuracy: 0.8200 - loss: 0.8918 - val_accuracy: 0.8625 - val_loss: 0.7510
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 131s 657ms/step - accuracy: 0.9217 - loss: 0.6564 - val_accuracy: 0.9422 - val_loss: 0.5614
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 139s 696ms/step - accuracy: 0.9695 - loss: 0.4932 - val_accuracy: 0.5922 - val_loss: 1.5966
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 126s 631ms/step - accuracy: 0.9523 - loss: 0.5231 - val_accuracy: 0.7937 - val_loss: 0.8966
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 178s 889ms/step - accuracy: 0.9702 - loss: 0.4721 - val_accuracy: 0.9547 - val_loss: 0.4738
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 119s 595ms/step - accuracy: 0.9716 - loss: 0.4432 - val_accuracy: 0.9734 - val_loss: 0.4236
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 164s 822ms/step - accuracy: 0.9584 -

# Model 3: CNN + BatchNormalization

In [24]:
# Model 3: CNN + BatchNormalization
model_3 = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])
model_3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_3 = model_3.fit(train_dataset, validation_data=val_dataset, epochs=10)

Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 141s 687ms/step - accuracy: 0.4955 - loss: 1.3668 - val_accuracy: 0.7297 - val_loss: 0.6357
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 127s 637ms/step - accuracy: 0.7994 - loss: 0.5138 - val_accuracy: 0.9328 - val_loss: 0.2385
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 131s 655ms/step - accuracy: 0.9344 - loss: 0.2040 - val_accuracy: 0.6875 - val_loss: 0.7548
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 132s 660ms/step - accuracy: 0.9724 - loss: 0.0971 - val_accuracy: 0.9344 - val_loss: 0.1842
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 146s 733ms/step - accuracy: 0.9807 - loss: 0.0704 - val_accuracy: 0.9625 - val_loss: 0.1013
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 153s 765ms/step - accuracy: 0.9880 - loss: 0.0444 - val_accuracy: 0.9422 - val_loss: 0.1469
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 136s 680ms/step - accuracy: 0.9841 - loss: 0.0534 - val_accuracy: 0.8641 - val_loss: 0.3738
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 136s 682ms/step - accuracy: 0.9729 -

# Model 4: CNN + L2 + BatchNormalization

In [25]:
# Model 4: CNN + L2 + BatchNormalization
model_4 = Sequential()
model_4.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(128, 128, 3)))
model_4.add(layers.BatchNormalization())
model_4.add(layers.MaxPooling2D((2, 2)))
model_4.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model_4.add(layers.BatchNormalization())
model_4.add(layers.MaxPooling2D((2, 2)))
model_4.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model_4.add(layers.BatchNormalization())
model_4.add(layers.MaxPooling2D((2, 2)))
model_4.add(layers.Flatten())
model_4.add(layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model_4.add(layers.BatchNormalization())
model_4.add(layers.Dropout(0.5))
model_4.add(layers.Dense(4, activation='softmax'))
model_4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_4 = model_4.fit(train_dataset, validation_data=val_dataset, epochs=10)


Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 155s 764ms/step - accuracy: 0.5135 - loss: 1.8051 - val_accuracy: 0.5641 - val_loss: 1.3442
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 127s 636ms/step - accuracy: 0.7895 - loss: 0.9632 - val_accuracy: 0.9062 - val_loss: 0.7056
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 126s 632ms/step - accuracy: 0.9119 - loss: 0.6723 - val_accuracy: 0.8391 - val_loss: 0.7934
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 141s 707ms/step - accuracy: 0.9524 - loss: 0.5548 - val_accuracy: 0.7469 - val_loss: 0.9331
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 140s 694ms/step - accuracy: 0.9520 - loss: 0.5314 - val_accuracy: 0.9297 - val_loss: 0.5591
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 127s 637ms/step - accuracy: 0.9520 - loss: 0.5319 - val_accuracy: 0.8734 - val_loss: 0.8288
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 110s 547ms/step - accuracy: 0.9568 - loss: 0.5262 - val_accuracy: 0.8234 - val_loss: 0.8380
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 157s 621ms/step - accuracy: 0.9605 -

# Model 5: CNN + Dropout

In [23]:
# Model 5: CNN + Dropout
model_5 = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])
model_5.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_5 = model_5.fit(train_dataset, validation_data=val_dataset, epochs=10)


Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 149s 730ms/step - accuracy: 0.5057 - loss: 1.3050 - val_accuracy: 0.1562 - val_loss: 6.0545
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 138s 690ms/step - accuracy: 0.8407 - loss: 0.4357 - val_accuracy: 0.7750 - val_loss: 0.4877
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 141s 703ms/step - accuracy: 0.9522 - loss: 0.1565 - val_accuracy: 0.8531 - val_loss: 0.3396
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 149s 743ms/step - accuracy: 0.9857 - loss: 0.0713 - val_accuracy: 0.9391 - val_loss: 0.1427
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 119s 597ms/step - accuracy: 0.9801 - loss: 0.0775 - val_accuracy: 0.7969 - val_loss: 0.5912
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 155s 778ms/step - accuracy: 0.9746 - loss: 0.0865 - val_accuracy: 0.9828 - val_loss: 0.0523
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 137s 682ms/step - accuracy: 0.9900 - loss: 0.0506 - val_accuracy: 0.9594 - val_loss: 0.0910
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 137s 682ms/step - accuracy: 0.9937 -

# Model 6: CNN + EarlyStopping

In [26]:
# Model 6: CNN + EarlyStopping
model_6 = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])
model_6.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history_6 = model_6.fit(train_dataset, validation_data=val_dataset, epochs=10, callbacks=[early_stopping])


Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 118s 580ms/step - accuracy: 0.5032 - loss: 1.3210 - val_accuracy: 0.7391 - val_loss: 0.5748
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 115s 574ms/step - accuracy: 0.8516 - loss: 0.3926 - val_accuracy: 0.9594 - val_loss: 0.1661
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 115s 576ms/step - accuracy: 0.9391 - loss: 0.1779 - val_accuracy: 0.5172 - val_loss: 3.9632
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 123s 616ms/step - accuracy: 0.9013 - loss: 0.2723 - val_accuracy: 0.8219 - val_loss: 0.4005
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 112s 559ms/step - accuracy: 0.9758 - loss: 0.0808 - val_accuracy: 0.6359 - val_loss: 1.1860


# Model 7: CNN + ReduceLROnPlateau

In [27]:
# Model 7: CNN + ReduceLROnPlateau
model_7 = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])
model_7.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
history_7 = model_7.fit(train_dataset, validation_data=val_dataset, epochs=10, callbacks=[reduce_lr])


Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 130s 642ms/step - accuracy: 0.4883 - loss: 1.3302 - val_accuracy: 0.6672 - val_loss: 0.7523 - learning_rate: 0.0010
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 120s 599ms/step - accuracy: 0.8269 - loss: 0.4561 - val_accuracy: 0.3609 - val_loss: 1.6711 - learning_rate: 0.0010
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 113s 566ms/step - accuracy: 0.9409 - loss: 0.1881 - val_accuracy: 0.9656 - val_loss: 0.1433 - learning_rate: 0.0010
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 124s 619ms/step - accuracy: 0.9664 - loss: 0.1040 - val_accuracy: 0.9563 - val_loss: 0.1166 - learning_rate: 0.0010
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 125s 625ms/step - accuracy: 0.9762 - loss: 0.0737 - val_accuracy: 0.9500 - val_loss: 0.1071 - learning_rate: 0.0010
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 149s 744ms/step - accuracy: 0.9866 - loss: 0.0536 - val_accuracy: 0.9750 - val_loss: 0.0689 - learning_rate: 0.0010
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 128s 639ms/step - accura

# Model 8: SVM with VGG16

In [ ]:
# Model 8: SVM with VGG16
from tensorflow.keras.applications import VGG16
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, f1_score

# Load VGG16 base model
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
vgg16_base.trainable = False

def extract_features(dataset, model):
    features, labels = [], []
    for images, labels_batch in dataset:
        # Extract features for the current batch
        batch_features = model.predict(images)
        features.append(batch_features)
        labels.append(labels_batch.numpy())

    # Combine all batch features and labels into single arrays
    features = np.vstack(features)
    labels = np.vstack(labels)

    # Reshape features to a 2D array (flatten each image feature map)
    features = features.reshape(features.shape[0], -1)

    # Convert labels to class indices
    labels = np.argmax(labels, axis=1)

    return features, labels

# Extract features and labels for train and test datasets
train_features, train_labels = extract_features(train_dataset, vgg16_base)
test_features, test_labels = extract_features(test_dataset, vgg16_base)

# Train the SVM model
svc_model = SVC(kernel='linear', class_weight='balanced')
svc_model.fit(train_features, train_labels)

# Make predictions
svc_predictions = svc_model.predict(test_features)

# Evaluate the SVM model
svc_accuracy = accuracy_score(test_labels, svc_predictions)
svc_f1 = f1_score(test_labels, svc_predictions, average='weighted')

# Print results
print("SVM Accuracy:", svc_accuracy)
print("SVM F1 Score:", svc_f1)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 

# Results Comparison

In [ ]:
# Store results for comparison
results = {
    "Model 1: Basic CNN": {"accuracy": history_1.history['val_accuracy'][-1]},
    "Model 2: CNN + L2": {"accuracy": history_2.history['val_accuracy'][-1]},
    "Model 3: CNN + BatchNormalization": {"accuracy": history_3.history['val_accuracy'][-1]},
    "Model 4: CNN + L2 + BatchNormalization": {"accuracy": history_4.history['val_accuracy'][-1]},
    "Model 5: CNN + Dropout": {"accuracy": history_5.history['val_accuracy'][-1]},
    "Model 6: CNN + EarlyStopping": {"accuracy": history_6.history['val_accuracy'][-1]},
    "Model 7: CNN + ReduceLROnPlateau": {"accuracy": history_7.history['val_accuracy'][-1]},
    "Model 8: SVM with VGG16 Features": {"accuracy": svc_accuracy},
}

# Plot results
names, accuracies = zip(*[(name, res["accuracy"]) for name, res in results.items()])
plt.barh(names, accuracies)
plt.xlabel("Accuracy")
plt.ylabel("Model")
plt.title("Model Accuracy Comparison")
plt.show()